In [3]:
import textwrap
from IPython.display import Markdown, display
from langchain_text_splitters import SentenceTransformersTokenTextSplitter
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np
import chromadb
import os
from chromadb.utils import embedding_functions
from chromadb import Client
from tqdm.notebook import trange, tqdm
from pprint import pprint



In [4]:
def convert_PDF_Text(pdf_path="/Users/harun/Documents/GitHub/AI-Powered-Regional-Cost-of-Living-Advisor/Search/Rag/Uni_fiyatları/halicUniversitesi.pdf"):
    reader = PdfReader(pdf_path)
    pdf_texts = [p.extract_text().strip() for p in reader.pages]
    # Filter the empty strings
    pdf_texts = [text for text in pdf_texts if text]
    return pdf_texts

pdf_texts=convert_PDF_Text()
pdf_texts

['Haliç Üniversitesi Eğitim Ücretleri ve Bursları 2024-2025\nHaliç Üniversitesinde eğitim-öğretim ücretlidir. 2024-2025 eğitim-öğretim yılı ücretleri,\nHalkla İlişkiler ve Tanıtım (%25 İndirimli) 213.000\x00    TL    (Ücretli) 284.000 TL\nGastronomi ve Mutfak Sanatları (%25 İndirimli) 270.000 TL (Ücretli) 360.000 TL\nTürk Dili ve Edebiyatı (% 50 İndirimli) 142.000 TL (Ücretli) 284.000 TL\nÇizgi Film ve Animasyon (%25 İndirimli) 231.000 TL (Ücretli) 308.000 TL\nGörsel İletişim Tasarımı (%25 İndirimli) 231.000 TL (Ücretli) 308.000 TL\nAmerikan Kültürü ve Edebiyatı (%25 İndirimli) 246.000 TL (Ücretli) 328.000 TL\nİngilizce Mütercim ve Tercümanlık (%25 İndirimli) 246.000 TL (Ücretli) 328.000 TL\nTekstil ve Moda Tasarımı (%25 İndirimli) 225.000 TL (Ücretli) 300.000 TL\nAntrenörlük Eğitimi (%25 İndirimli) 240.750 TL (Ücretli) 321.000 TL\nSpor Yöneticiliği (%25 İndirimli) 231.000 TL (Ücretli) 308.000 TL\nSpor Yöneticiliği (İngilizce) (%25 İndirimli) 231.000 TL (Ücretli) 308.000 TL\nBeden Eğit

In [5]:
# Helper for displaying text
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))



In [7]:
def text_Chunks_in_Char(pdf_texts, chunk_size=800, chunk_overlap=100):
    # Combine all PDF texts into one string
    full_text = '\n\n'.join(pdf_texts)

    # Initialize sentence-based text splitter
    sentence_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ".", ":", ";", ",", " ", ""],  # Process in this order
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )

    # Split text into chunks based on sentences
    chunks = sentence_splitter.split_text(full_text)
    return chunks
chunks=text_Chunks_in_Char(pdf_texts)
chunks[2]

'Psikoloji (%25 İndirimli) 258.000 TL (Ücretli) 344.000 TL\nPsikoloji (İngilizce) (%25 İndirimli) 270.000 TL (Ücretli) 360.000 TL\nUluslararası Ticaret ve Finans (İngilizce) (%25 İndirimli) 213.000 TL (Ücretli) 284.000 TL\nSiyaset Bilimi ve Uluslararası İlişkiler (%25 İndirimli) 213.000 TL (Ücretli) 284.000 TL\nİşletme (%25 İndirimli) 213.000 TL (Ücretli) 284.000 TL\nİşletme (İngilizce) (%25 İndirimli) 213.000 TL (Ücretli) 284.000 TL\nYönetim Bilişim Sistemleri (İngilizce) (%25 İndirimli) 240.000 TL (Ücretli) 320.000 TL'

In [8]:
sentence_transformer_model = "distiluse-base-multilingual-cased-v1"

In [9]:

def convert_Chunk_Token(text_chunksinChar, sentence_transformer_model, chunk_overlap=10, tokens_per_chunk=128):
    token_splitter = SentenceTransformersTokenTextSplitter(
        chunk_overlap=chunk_overlap,
        model_name=sentence_transformer_model,
        tokens_per_chunk=tokens_per_chunk)

    text_chunksinTokens = []
    for text in text_chunksinChar:
        text_chunksinTokens += token_splitter.split_text(text)
    print(f"\nTotal number of chunks (document splited by 128 tokens per chunk): {len(text_chunksinTokens)}")
    return text_chunksinTokens
tokens=convert_Chunk_Token(chunks,sentence_transformer_model)
tokens[7]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Total number of chunks (document splited by 128 tokens per chunk): 14


'TL İşletme ( % 25 İndirimli ) 213. 000 TL ( Ücretli ) 284. 000 TL İşletme ( İngilizce ) ( % 25 İndirimli ) 213. 000 TL ( Ücretli ) 284. 000 TL Yönetim Bilişim Sistemleri ( İngilizce ) ( % 25 İndirimli ) 240. 000 TL ( Ücretli ) 320. 000 TL'

In [53]:
sentence_transformer_model = "distiluse-base-multilingual-cased-v1"
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=sentence_transformer_model)


In [11]:
def create_chroma_client(collection_name, embedding_function):
    chroma_client = chromadb.PersistentClient()

    # Check if collection already exists and delete it
    existing_collections = [col.name for col in chroma_client.list_collections()]
    if collection_name in existing_collections:
        chroma_client.delete_collection(collection_name)

    # Create a new collection
    chroma_collection = chroma_client.create_collection(
        name=collection_name,
        embedding_function=embedding_function
    )

    return chroma_client, chroma_collection



In [12]:
def add_meta_data(chunks, title, category, initial_id):
    ids = [str(i + initial_id) for i in range(len(chunks))]
    filename = os.path.basename(title)
    metadata = {
        'document': filename,
        'category': category
    }
    metadatas = [metadata for i in range(len(chunks))]
    return ids, metadatas


In [13]:
def add_document_to_collection(ids, metadatas, chunks, chroma_collection):
    chroma_collection.upsert(ids=ids, metadatas=metadatas, documents=chunks)
    return chroma_collection

In [15]:
def retrieveDocs(chroma_collection, query, file=None, n_results=10, return_only_docs=False):

    # Build query parameters
    query_params = {
        "query_texts": [query],
        "include": ["documents", "metadatas", 'distances'],
        "n_results": n_results
    }

    # Add file filter if provided
    if file is not None:
        query_params["where"] = {"document": f"{file}.pdf"}

    # Execute the query
    results = chroma_collection.query(**query_params)

    if return_only_docs:
        return results['documents'][0]
    else:
        return results


In [20]:
def show_results(results, return_only_docs=False):
    if return_only_docs:
        retrieved_documents = results
        if len(retrieved_documents) == 0:
            print("No results found.")
            return
        for i, doc in enumerate(retrieved_documents):
            print(f"Document {i + 1}:")
            print("\tDocument Text: ")
            pprint(doc)
    else:
        retrieved_documents = results['documents'][0]
        if len(retrieved_documents) == 0:
            print("No results found.")
            return
        retrieved_documents_metadata = results['metadatas'][0]
        retrieved_documents_distances = results['distances'][0]
        print("------- Retrieved documents -------\n")

        for i, doc in enumerate(retrieved_documents):
            print(f"Document {i + 1}:")
            print("\tDocument Text: ")
            pprint(doc)
            # Extract filename from the full path
            full_path = retrieved_documents_metadata[i]['document']
            filename = os.path.basename(full_path)
            print(f"\tDocument Source: {filename}")
            print(f"\tDocument Source Type: {retrieved_documents_metadata[i]['category']}")
            print(f"\tDocument Distance: {retrieved_documents_distances[i]}")
            print("-" * 80)


In [21]:

def list_files_in_directory(directory_path):
    all_entries = os.listdir(directory_path)

    files_only = [entry for entry in all_entries
                  if os.path.isfile(os.path.join(directory_path, entry))]
    return files_only


In [22]:
def get_all_pdf_paths(
        directory_path="/Users/harun/Documents/GitHub/AI-Powered-Regional-Cost-of-Living-Advisor/Search/Rag/Uni_fiyatları"):
    """
    Get all PDF file paths from the specified directory.

    Args:
        directory_path (str): Path to the directory containing PDF files

    Returns:
        list: List of full paths to all PDF files in the directory
    """
    all_files = list_files_in_directory(directory_path)
    pdf_files = [f for f in all_files if f.lower().endswith('.pdf')]
    full_paths = [os.path.join(directory_path, pdf_file) for pdf_file in pdf_files]
    return full_paths


In [24]:
def load_multiple_pdfs_to_ChromaDB(collection_name, sentence_transformer_model):
    """
    Load multiple PDFs into ChromaDB collection with proper chunking and embedding.

    Args:
        collection_name (str): Name of the ChromaDB collection
        sentence_transformer_model (str): Name of the sentence transformer model to use

    Returns:
        tuple: (chroma_client, chroma_collection)
    """
    # Initialize embedding function
    embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=sentence_transformer_model)

    # Create or get ChromaDB client and collection
    chroma_client, chroma_collection = create_chroma_client(collection_name, embedding_function)

    # Get current ID for continuous indexing
    current_id = chroma_collection.count()

    # Get all PDF paths from the directory
    pdf_paths = get_all_pdf_paths()

    for pdf_path in pdf_paths:
        # Extract text from PDF
        pdf_texts = convert_PDF_Text(pdf_path)

        # Create character-based chunks first
        char_chunks = text_Chunks_in_Char(pdf_texts)
        
        # Convert to token-based chunks
        token_chunks = convert_Chunk_Token(char_chunks, sentence_transformer_model)

        # Add metadata and get IDs
        ids, metadatas = add_meta_data(token_chunks, pdf_path, "PricePaper", current_id)

        # Update current_id for next document
        current_id += len(token_chunks)

        # Add to collection
        chroma_collection = add_document_to_collection(ids, metadatas, token_chunks, chroma_collection)

    return chroma_client, chroma_collection

In [25]:
def get_existing_chroma_collection(collection_name):

    chroma_client = chromadb.PersistentClient()
    
    # Get the existing collection
    chroma_collection = chroma_client.get_collection(
        name=collection_name
    )
    
    return  chroma_collection

In [26]:
from KeywordAgent import keyword_agent

# Test code
chroma_client, chroma_collection = load_multiple_pdfs_to_ChromaDB("UniPrices", sentence_transformer_model)
#chroma_collection=get_existing_chroma_collection("UniPrices")

query = "Haliç Üniversitesi Tiyatro"
keyword = keyword_agent.parse_keywords(query)
filtered_keyword = keyword.strip('"')
if filtered_keyword != 'None':
    retrieved_documents = retrieveDocs(chroma_collection, query, "halicUniversitesi", 10)
    show_results(retrieved_documents)
else:
    retrieved_documents = retrieveDocs(chroma_collection, query, None, 10)
    show_results(retrieved_documents)




Total number of chunks (document splited by 128 tokens per chunk): 1

Total number of chunks (document splited by 128 tokens per chunk): 16

Total number of chunks (document splited by 128 tokens per chunk): 28

Total number of chunks (document splited by 128 tokens per chunk): 16

Total number of chunks (document splited by 128 tokens per chunk): 20

Total number of chunks (document splited by 128 tokens per chunk): 9

Total number of chunks (document splited by 128 tokens per chunk): 23

Total number of chunks (document splited by 128 tokens per chunk): 11

Total number of chunks (document splited by 128 tokens per chunk): 18

Total number of chunks (document splited by 128 tokens per chunk): 18

Total number of chunks (document splited by 128 tokens per chunk): 15

Total number of chunks (document splited by 128 tokens per chunk): 66

Total number of chunks (document splited by 128 tokens per chunk): 1

Total number of chunks (document splited by 128 tokens per chunk): 1

Total num

AttributeError: 'KeywordAgent' object has no attribute 'parse_keywords'